In [1]:
# Download and prepare the data
!bash venv/lib/python3.8/site-packages/fairseq/examples/translation/prepare-iwslt14.sh

Cloning Moses github repository (for tokenization scripts)...
Cloning into 'mosesdecoder'...
remote: Enumerating objects: 148464, done.
remote: Counting objects: 100% (892/892), done.
remote: Compressing objects: 100% (367/367), done.
remote: Total 148464 (delta 567), reused 802 (delta 521), pack-reused 147572 (from 1)
Receiving objects: 100% (148464/148464), 129.99 MiB | 13.17 MiB/s, done.
Resolving deltas: 100% (114593/114593), done.
Cloning Subword NMT repository (for BPE pre-processing)...
Cloning into 'subword-nmt'...
remote: Enumerating objects: 622, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 622 (delta 25), reused 31 (delta 16), pack-reused 576 (from 1)
Receiving objects: 100% (622/622), 261.27 KiB | 1.45 MiB/s, done.
Resolving deltas: 100% (374/374), done.
--2025-01-22 13:14:42--  https://wit3.fbk.eu/archive/2014-01/texts/de/en/de-en.tgz
Resolving wit3.fbk.eu (wit3.fbk.eu)... 142.250.74.147, 2a00:1450:400f:

In [2]:
# Preprocess/binarize the data
!TEXT=examples/translation/iwslt14.tokenized.de-en && \
fairseq-preprocess --source-lang de --target-lang en \
    --trainpref $TEXT/train --validpref $TEXT/valid --testpref $TEXT/test \
    --destdir data-bin/iwslt14.tokenized.de-en \
    --workers 20

2025-01-21 20:49:13 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2025-01-21 20:49:13 | INFO | fairseq_cli.preprocess | Namespace(aim_repo=None, aim_run_hash=None, align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin/iwslt14.tokenized.de-en', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tm

In [7]:
# train a Transformer translation model over this data
!CUDA_VISIBLE_DEVICES=0 fairseq-train \
    data-bin/iwslt14.tokenized.de-en \
    --arch transformer_iwslt_de_en --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096

2025-01-22 10:23:14 | INFO | fairseq_cli.train | Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='transformer_iwslt_de_en', attention_dropout=0.0, batch_size=None, batch_size_valid=None, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='data-bin/iwslt14.tokenized.de-en', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=4, decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=1024, decoder_input_dim=512, decoder_layerdrop=0, decoder_layers=6, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_before=False, decoder_output_dim=512, device_id=0, disab

In [2]:
!pip list | grep fairseq

fairseq                  0.10.2

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
# modified to align with hyperparameter from paper
# train a Transformer translation model over this data
!CUDA_VISIBLE_DEVICES=0 fairseq-train \
    data-bin/iwslt14.tokenized.de-en \
    --arch transformer_iwslt_de_en --share-decoder-input-output-embed \
    --criterion label_smoothed_cross_entropy \
    --max-tokens 4096 \
    --max-update 500_000

Traceback (most recent call last):
  File "/home/sondre/fairseq/venv/bin/fairseq-train", line 5, in <module>
    from fairseq_cli.train import cli_main
  File "/home/sondre/fairseq/venv/lib/python3.8/site-packages/fairseq_cli/train.py", line 19, in <module>
    from fairseq import (
  File "/home/sondre/fairseq/venv/lib/python3.8/site-packages/fairseq/__init__.py", line 19, in <module>
    import fairseq.criterions  # noqa
  File "/home/sondre/fairseq/venv/lib/python3.8/site-packages/fairseq/criterions/__init__.py", line 13, in <module>
    from fairseq.criterions.fairseq_criterion import (  # noqa
  File "/home/sondre/fairseq/venv/lib/python3.8/site-packages/fairseq/criterions/fairseq_criterion.py", line 9, in <module>
    from fairseq import metrics, utils
  File "/home/sondre/fairseq/venv/lib/python3.8/site-packages/fairseq/utils.py", line 20, in <module>
    from fairseq.data import iterators
  File "/home/sondre/fairseq/venv/lib/python3.8/site-packages/fairseq/data/__init__.py", l

In [10]:
# train a Transformer translation model over this data
!CUDA_VISIBLE_DEVICES=0 python3.8 -m fairseq_cli.train \
    data-bin/iwslt14.tokenized.de-en \
    --arch transformer_iwslt_de_en --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096

Traceback (most recent call last):
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/sondre/fairseq/fairseq_cli/train.py", line 30, in <module>
    from fairseq import checkpoint_utils, options, quantization_utils, tasks, utils
  File "/home/sondre/fairseq/fairseq/__init__.py", line 33, in <module>
    import fairseq.criterions  # noqa
  File "/home/sondre/fairseq/fairseq/criterions/__init__.py", line 36, in <module>
    importlib.import_module("fairseq.criterions." + file_name)
  File "/usr/lib/python3.8/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "/home/sondre/fairseq/fairseq/criterions/ctc.py", line 20, in <module>
    from fairseq.data.data_utils import post_process
  File "/home/sondre/fairseq/fairseq/data/__init__.py", line 24, in <modu